In [5]:
import urllib.request
import os 
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
tmpDirectory = os.getcwd() +"/tmp"
if not os.path.exists(tmpDirectory):
        os.makedirs(tmpDirectory)
filename = tmpDirectory+"/cats_and_dogs_filtered.zip"
urllib.request.urlretrieve(url,filename)

('/home/tioner/Documentos/maestria/2/sistemas cognitivos/image_secuencial_clasifier/tmp/cats_and_dogs_filtered.zip',
 <http.client.HTTPMessage at 0x7f448c1f7850>)

In [7]:
import zipfile
zipRef = zipfile.ZipFile(filename,"r")

zipRef.extractall(tmpDirectory)
zipRef.close()